In [18]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import random
import openpyxl

servico = Service(ChromeDriverManager().install())
navegador = webdriver.Chrome(service=servico)

# Projeto robô automático que dispara mensagem no WhatsApp. As mensagens devem ser enviadas para os clientes que estão
# em uma base de dados na Planilha. O robô iniciará a conversa com um cliente e digitará um texto, que será retirado
# de uma lista feita, contendo mais de 5 mensagens. O robô escolherá uma aleatória e escreverá para o cliente usando pausas
# na hora de digitar para simular a interação humana.
# O robô lerá a planilha e pegará o nome do cliente e o número do cliente para enviar a mensagem.
# O robô enviará a mensagem para o cliente e esperará 10 segundos antes de enviar a próxima mensagem para o próximo cliente.
# O robô escreverá, dando uma pausa de 1 segundo para cada letra digitada para simular a interação humana.

# Abrindo o navegador
navegador.get('https://web.whatsapp.com/')
time.sleep(30)  # Aumentando o tempo para garantir que o usuário tenha tempo de fazer login

# Abrindo a planilha com openpyxl
planilha = openpyxl.load_workbook('bot_numeros.xlsx')

# Detectando automaticamente o nome da folha
nome_folha = planilha.active.title

# Lendo a planilha com openpyxl
for linha in planilha[nome_folha]:
    # Separando o nome do número
    nome = linha[0].value
    numero = linha[1].value

    # Esperando até que o campo de pesquisa esteja disponível
    campo_pesquisa_xpath = '//*[@id="side"]/div[1]/div/div[2]/div[2]/div/div[1]/p'
    WebDriverWait(navegador, 60).until(EC.presence_of_element_located((By.XPATH, campo_pesquisa_xpath)))

    # Abrindo a conversa com o cliente
    campo_pesquisa = navegador.find_element(By.XPATH, campo_pesquisa_xpath)
    campo_pesquisa.click()
    campo_pesquisa.send_keys(numero)
    campo_pesquisa.send_keys(Keys.RETURN)  # Pressiona Enter para iniciar a conversa
    time.sleep(5)  # Tempo adicional para garantir que a conversa esteja carregada

    # Atualizando o campo de mensagem XPath
    campo_mensagem_xpath = '//*[@id="main"]/footer/div[1]/div/span[2]/div/div[2]/div[1]/div/div[1]/p'

    # Esperando até que o campo de mensagem esteja clicável
    campo_mensagem = WebDriverWait(navegador, 20).until(EC.element_to_be_clickable((By.XPATH, campo_mensagem_xpath)))

    # Clicando no campo de digitação
    campo_mensagem.click()
    time.sleep(2)

    # Criando a lista de mensagens
    lista_mensagens = ['Olá, tudo bem?', 'Como vai?', 'Estou entrando em contato...', 'O produto é muito bom...', 'Estou enviando o link do produto...', 'Se tiver alguma dúvida...']

    # Escolhendo a mensagem aleatória
    mensagem = random.choice(lista_mensagens)

    # Tentando enviar a mensagem até que seja bem-sucedido
    while True:
        try:
            # Digitando a mensagem
            campo_mensagem.send_keys(mensagem)
            time.sleep(1)

            # Enviando a mensagem pressionando Enter
            campo_mensagem.send_keys(Keys.RETURN)
            break  # Saindo do loop se a mensagem for enviada com sucesso
        except Exception as e:
            print(f"Erro ao enviar mensagem: {e}")
            time.sleep(2)  # Aguardando 2 segundos antes de tentar novamente

    time.sleep(10)  # Tempo adicional após o envio da mensagem
